<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Imp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧:
Scenario:
You are working as a Data Engineer for a retail company. The company has provided you with customer transaction data in a Python dictionary.
Your task is to create a PySpark DataFrame from the given dictionary and answer the following:
Display the schema of the created DataFrame.
Filter the DataFrame to show only transactions where the amount is greater than 100.
Add a new column discounted_amount that applies a 10% discount to all transactions.

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
transaction_data = [
 {"transaction_id": 1, "customer_id": 101, "amount": 150, "date": "2025-01-01"},
 {"transaction_id": 2, "customer_id": 102, "amount": 90, "date": "2025-01-02"},
 {"transaction_id": 3, "customer_id": 103, "amount": 200, "date": "2025-01-03"},
 {"transaction_id": 4, "customer_id": 104, "amount": 50, "date": "2025-01-04"},
 {"transaction_id": 5, "customer_id": 105, "amount": 120, "date": "2025-01-05"}
]


In [6]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName('learning').getOrCreate()

data=transaction_data = [
 {"transaction_id": 1, "customer_id": 101, "amount": 150, "date": "2025-01-01"},
 {"transaction_id": 2, "customer_id": 102, "amount": 90, "date": "2025-01-02"},
 {"transaction_id": 3, "customer_id": 103, "amount": 200, "date": "2025-01-03"},
 {"transaction_id": 4, "customer_id": 104, "amount": 50, "date": "2025-01-04"},
 {"transaction_id": 5, "customer_id": 105, "amount": 120, "date": "2025-01-05"}
]

df=spark.createDataFrame(data)

df.printSchema()

df.show()

df1=df.filter(col('amount') > 100 )

df1.show()

df2=df.withColumn('discounted_amount',col('amount')*0.9)

df2.show()



root
 |-- amount: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- transaction_id: long (nullable = true)

+------+-----------+----------+--------------+
|amount|customer_id|      date|transaction_id|
+------+-----------+----------+--------------+
|   150|        101|2025-01-01|             1|
|    90|        102|2025-01-02|             2|
|   200|        103|2025-01-03|             3|
|    50|        104|2025-01-04|             4|
|   120|        105|2025-01-05|             5|
+------+-----------+----------+--------------+

+------+-----------+----------+--------------+
|amount|customer_id|      date|transaction_id|
+------+-----------+----------+--------------+
|   150|        101|2025-01-01|             1|
|   200|        103|2025-01-03|             3|
|   120|        105|2025-01-05|             5|
+------+-----------+----------+--------------+

+------+-----------+----------+--------------+-----------------+
|amount|customer_i